In [1]:
# !pip install bs4==0.0.1
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time
import os
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')



import warnings
warnings.filterwarnings('ignore')

## 1. Crawling Data

### 1.1 crawling Stadiums data and matches data

In [3]:
def crawl_stadium(season_start_year=1992):

    international_history = 'https://www.worldfootball.net/venues/eng-premier-league-{}-{}/'.format(season_start_year, season_start_year+1)
    page = requests.get(international_history)
    soup = BeautifulSoup(page.content, 'html.parser')

    table = soup.findAll('table')[0]
    tds = table.find_all('td')

    data = []
    for row_index in range(int(len(tds)/6)):
        
        current_position = row_index * 6

        stadium_name = tds[current_position + 1].text
        
        href = list(tds[current_position + 1].children)[0]['href']
        city = tds[current_position + 2].text
        capacity = tds[current_position + 5].text

        data.append((stadium_name, href, city, capacity))

    data = pd.DataFrame(data, columns=['stadium_name', 'href', 'city', 'capacity'])
    data.to_csv('datasets/stadiums/{}-{}_stadiums.csv'.format(season_start_year, season_start_year+1))
    print('finish {}-{}'.format(season_start_year, season_start_year+1))


# matches data
def crawl_matches_by_stadium(href, year, nth):

    url = 'https://www.worldfootball.net{}{}/2/'.format(href, year)
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')

    table = soup.findAll('table')[1]
    tds = table.find_all('td')

    data = []
    for row_index in range(int(len(tds)/7)):
        
        current_position = row_index * 7

        match_type = tds[current_position + 0].text
        date = tds[current_position + 1].text
        home_team = tds[current_position + 2].a['href'].split('/')[2] if tds[current_position + 2].a else tds[current_position + 2].text
        away_team = tds[current_position + 4].a['href'].split('/')[2] if tds[current_position + 4].a else tds[current_position + 4].text
        result = tds[current_position + 5].text
        capacity = tds[current_position + 6].text

        data.append((match_type, date, home_team, away_team, result, capacity))

    season = '{}-{}'.format(1991+nth, 1992+nth)
    stadium = href.split('/')[2]
    
    data = pd.DataFrame(data, columns=['match_type', 'date', 'home_team', 'away_team', 'result', 'capacity'])
    if os.path.isdir('datasets/matches/{}/'.format(season)) == False:
        os.makedirs('datasets/matches/{}/'.format(season))
    path = 'datasets/matches/{}/{}_{}.csv'.format(season, stadium, year)
    data.to_csv(path)
    print('finish {}'.format(path))

In [ ]:
# crawl each season stasium data
for i in range(1992, 2022):
    crawl_stadium(i)
    time.sleep(2)
    
# get all data
for filename in os.listdir('datasets/stadiums'):
    stadiums = pd.read_csv('datasets/stadiums/{}'.format(filename))
    for i, row in stadiums.iterrows():
        year1, year2 = filename.split('_')[0].split('-')
        crawl_matches_by_stadium(row['href'], year1, int(year1)-1991)
        time.sleep(3)
        crawl_matches_by_stadium(row['href'], year2, int(year1)-1991)
        time.sleep(3)


In [5]:
# update 2022 data
def update2022():
    crawl_stadium(2021)
    stadiums = pd.read_csv('datasets/stadiums/2021-2022_stadiums.csv')
    for i, row in stadiums.iterrows():
        crawl_matches_by_stadium(row['href'], '2022', int(2021)-1991)
        time.sleep(3)
        
update2022()

finish 2021-2022
finish datasets/matches/2021-2022/amex-stadium-brighton_2022.csv
finish datasets/matches/2021-2022/anfield-liverpool_2022.csv
finish datasets/matches/2021-2022/brentford-community-stadium-brentford_2022.csv
finish datasets/matches/2021-2022/carrow-road-norwich_2022.csv
finish datasets/matches/2021-2022/elland-road-leeds_2022.csv
finish datasets/matches/2021-2022/emirates-stadium-london_2022.csv
finish datasets/matches/2021-2022/etihad-stadium-manchester_2022.csv
finish datasets/matches/2021-2022/goodison-park-liverpool_2022.csv
finish datasets/matches/2021-2022/king-power-stadium-leicester_2022.csv
finish datasets/matches/2021-2022/london-stadium-london_2022.csv
finish datasets/matches/2021-2022/molineux-stadium-wolverhampton_2022.csv
finish datasets/matches/2021-2022/old-trafford-manchester_2022.csv
finish datasets/matches/2021-2022/saint-marys-southampton_2022.csv
finish datasets/matches/2021-2022/selhurst-park-london_2022.csv
finish datasets/matches/2021-2022/st-jam